In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('../lista_5/dataset/mensagens_com_frequencia_e_texto_processado.csv')
df.shape

(188264, 4)

In [3]:
df.head()

,texto_original,frequencia,texto_limpo,text_processed
0,▪️ 01/11/2022 - 08:00 \n\nÁsia-Pacífico (fecha...,10,01112022 0800 asiapacifico fechado spasx 165 6...,01112022 0800 asiapacifico fechado spasx 165 6...
1,*02.10.2022 TSE - Governo-SP às 18h 59m 00s*,8,02102022 tse governosp as 18h 59m 00s,02102022 tse governosp 18h 59m 00s
2,02.10.2022 TSE - Governo-SP às 20h 10m 35s,4,02102022 tse governosp as 20h 10m 35s,02102022 tse governosp 20h 10m 35s
3,02.10.2022 TSE - Presidente - às 18h 55m 31s,10,02102022 tse presidente as 18h 55m 31s,02102022 tse presidente 18h 55m 31s
4,*02.10.2022 TSE - Presidente às 20h 02m 45s*,3,02102022 tse presidente as 20h 02m 45s,02102022 tse presidente 20h 02m 45s


In [3]:
# Excluir as linhas onde 'text_processed' é nula, modificando o dataframe original
df.dropna(subset=['text_processed'], inplace=True)
df.shape

(188263, 4)

In [4]:
df = df[df['text_processed'].str.strip().str.split().apply(lambda x: len([w for w in x if w])) >= 5]
df.shape

(183300, 4)

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np

# --- Codificando os textos ---
modelo_bert = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

df = df.reset_index(drop=True)

batch_size = 5000
vetores = []
print("Gerando embeddings em batches...")
for i in tqdm(range(0, len(df), batch_size)):
    batch_textos = df['text_processed'].iloc[i:i + batch_size].tolist()
    batch_vetores = modelo_bert.encode(batch_textos, show_progress_bar=False)
    vetores.extend(batch_vetores)

df['vector'] = vetores



# Parâmetro de similaridade
limiar_similaridade = 0.90
janela = 1000  # K Vizinhos mais proxímos

# Inicialização
indices_para_remover = set()
agrupamento = defaultdict(list)

print(" Comparando vetores por similaridade...")

# Comparar vetores usando cosine_similarity em janela
for i in tqdm(range(len(df))):
    if i in indices_para_remover:
        continue
    vec_i = np.array(df.iloc[i]['vector']).reshape(1, -1)
    for j in range(i + 1, min(i + janela, len(df))):
        if j in indices_para_remover:
            continue
        vec_j = np.array(df.iloc[j]['vector']).reshape(1, -1)
        sim = cosine_similarity(vec_i, vec_j)[0][0]
        if sim >= limiar_similaridade:
            agrupamento[i].append(j)
            indices_para_remover.add(j)

# Soma frequência das mensagens agrupadas
linhas_resultado = []
mensagens_utilizadas = set()

for idx_base, similares_idxs in agrupamento.items():
    if idx_base in mensagens_utilizadas:
        continue
    freq_total = df.iloc[idx_base]['frequencia']
    mensagens_utilizadas.add(idx_base)

    for sim_idx in similares_idxs:
        freq_total += df.iloc[sim_idx]['frequencia']
        mensagens_utilizadas.add(sim_idx)

    linha = df.iloc[idx_base].copy()
    linha['frequencia'] = freq_total
    linhas_resultado.append(linha)

# Mensagens que não foram agrupadas
indices_restantes = set(range(len(df))) - mensagens_utilizadas
df_restantes = df.iloc[list(indices_restantes)].copy()

# Resultado final
df_resultado = pd.DataFrame(linhas_resultado)
df_final_total = pd.concat([df_resultado, df_restantes], ignore_index=True)

print(f"\n Total de mensagens finais: {len(df_final_total)}")
print(f"  Mensagens removidas por similaridade: {len(indices_para_remover)}")

# --- Criar DataFrame com mensagens similares lado a lado ---
pares_similares = []

for idx_base, similares_idxs in agrupamento.items():
    texto_base = df.at[idx_base, 'text_processed']
    vetor_base = df.at[idx_base, 'vector']
    freq_base = df.at[idx_base, 'frequencia']

    for sim_idx in similares_idxs:
        texto_similar = df.at[sim_idx, 'text_processed']
        vetor_similar = df.at[sim_idx, 'vector']
        freq_similar = df.at[sim_idx, 'frequencia']
        similaridade = cosine_similarity(
            np.array(vetor_base).reshape(1, -1),
            np.array(vetor_similar).reshape(1, -1)
        )[0][0]

        if similaridade >= limiar_similaridade:
          pares_similares.append({
                'mensagem_base': texto_base,
                'frequencia_base': freq_base,
                'mensagem_similar': texto_similar,
                'frequencia_similar': freq_similar,
                'similaridade': round(similaridade, 4)
            })

# Criar o DataFrame final com os pares similares
df_pares_similares = pd.DataFrame(pares_similares)

print(f"\nTotal de pares similares encontrados: {len(df_pares_similares)}")



c:\Users\Melissa Felipe\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Gerando embeddings em batches...


100%|██████████| 37/37 [1:33:19<00:00, 151.35s/it]


 Comparando vetores por similaridade...


100%|██████████| 183300/183300 [44:02:46<00:00,  1.16it/s]        



 Total de mensagens finais: 175436
  Mensagens removidas por similaridade: 7864

Total de pares similares encontrados: 7864


In [6]:
df_pares_similares.head(10)

,mensagem_base,frequencia_base,mensagem_similar,frequencia_similar,similaridade
0,01112022 0800 asiapacifico fechado spasx 165 6...,10,11102022 0800 asiapacifico fechado spasx 034 6...,8,0.9853
1,01112022 0800 asiapacifico fechado spasx 165 6...,10,11112022 0800 asiapacifico fechado spasx 279 7...,3,0.9688
2,01112022 0800 asiapacifico fechado spasx 165 6...,10,14102022 0800 asiapacifico fechado spasx 175 6...,3,0.9840
3,02102022 tse governosp 18h 59m 00s,8,02102022 tse governosp 20h 10m 35s,4,0.9196
4,02102022 tse presidente 18h 55m 31s,10,02102022 tse presidente 20h 02m 45s,3,0.9730
5,02102022 tse presidente 18h 55m 31s,10,02102022 tse presidente 22h 21m 36s,2,0.9672
6,0516 02112022 ricardo jornalista 2 entenda dif...,125,111 0710 sub ten picanco ferreira entenda dife...,39,0.9824
7,08112022 0000 asiapacifico spasx 025 695080 ni...,5,abertura 01112022 2240 abertura asiapacifico s...,10,0.9260
8,08112022 0000 asiapacifico spasx 025 695080 ni...,5,abertura 10102022 2240 abertura asiapacifico s...,4,0.9024
9,08112022 0000 asiapacifico spasx 025 695080 ni...,5,abertura 20102022 2240 abertura asiapacifico s...,2,0.9058


In [7]:
df_final_total.shape

(175436, 5)

## Segunda rodada

In [7]:
df_segunda_rodada = df_final_total.copy()

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np

# --- Codificando os textos ---
modelo_bert = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

df_segunda_rodada = df_segunda_rodada.reset_index(drop=True)

batch_size = 5000
vetores = []
print("Gerando embeddings em batches...")
for i in tqdm(range(0, len(df_segunda_rodada), batch_size)):
    batch_textos = df_segunda_rodada['text_processed'].iloc[i:i + batch_size].tolist()
    batch_vetores = modelo_bert.encode(batch_textos, show_progress_bar=False)
    vetores.extend(batch_vetores)

df_segunda_rodada['vector'] = vetores



# Parâmetro de similaridade
limiar_similaridade = 0.90
janela = 1000  # reduz custo computacional

# Inicialização
indices_para_remover = set()
agrupamento = defaultdict(list)

print(" Comparando vetores por similaridade...")

# Comparar vetores usando cosine_similarity em janela
for i in tqdm(range(len(df_segunda_rodada))):
    if i in indices_para_remover:
        continue
    vec_i = np.array(df_segunda_rodada.iloc[i]['vector']).reshape(1, -1)
    for j in range(i + 1, min(i + janela, len(df_segunda_rodada))):
        if j in indices_para_remover:
            continue
        vec_j = np.array(df_segunda_rodada.iloc[j]['vector']).reshape(1, -1)
        sim = cosine_similarity(vec_i, vec_j)[0][0]
        if sim >= limiar_similaridade:
            agrupamento[i].append(j)
            indices_para_remover.add(j)

# Soma frequência das mensagens agrupadas
linhas_resultado = []
mensagens_utilizadas = set()

for idx_base, similares_idxs in agrupamento.items():
    if idx_base in mensagens_utilizadas:
        continue
    freq_total = df_segunda_rodada.iloc[idx_base]['frequencia']
    mensagens_utilizadas.add(idx_base)

    for sim_idx in similares_idxs:
        freq_total += df_segunda_rodada.iloc[sim_idx]['frequencia']
        mensagens_utilizadas.add(sim_idx)

    linha = df_segunda_rodada.iloc[idx_base].copy()
    linha['frequencia'] = freq_total
    linhas_resultado.append(linha)

# Mensagens que não foram agrupadas
indices_restantes = set(range(len(df_segunda_rodada))) - mensagens_utilizadas
df_restantes = df_segunda_rodada.iloc[list(indices_restantes)].copy()

# Resultado final
df_resultado = pd.DataFrame(linhas_resultado)
df_final_total = pd.concat([df_resultado, df_restantes], ignore_index=True)

print(f"\n Total de mensagens finais: {len(df_final_total)}")
print(f"  Mensagens removidas por similaridade: {len(indices_para_remover)}")

# --- Criar DataFrame com mensagens similares lado a lado ---
pares_similares = []

for idx_base, similares_idxs in agrupamento.items():
    texto_base = df_segunda_rodada.at[idx_base, 'text_processed']
    vetor_base = df_segunda_rodada.at[idx_base, 'vector']
    freq_base = df_segunda_rodada.at[idx_base, 'frequencia']

    for sim_idx in similares_idxs:
        texto_similar = df_segunda_rodada.at[sim_idx, 'text_processed']
        vetor_similar = df_segunda_rodada.at[sim_idx, 'vector']
        freq_similar = df_segunda_rodada.at[sim_idx, 'frequencia']
        similaridade = cosine_similarity(
            np.array(vetor_base).reshape(1, -1),
            np.array(vetor_similar).reshape(1, -1)
        )[0][0]

        if similaridade >= limiar_similaridade:
          pares_similares.append({
                'mensagem_base': texto_base,
                'frequencia_base': freq_base,
                'mensagem_similar': texto_similar,
                'frequencia_similar': freq_similar,
                'similaridade': round(similaridade, 4)
            })

# Criar o DataFrame final com os pares similares
df_pares_similares = pd.DataFrame(pares_similares)

print(f"\nTotal de pares similares encontrados: {len(df_pares_similares)}")



Gerando embeddings em batches...


100%|██████████| 2/2 [00:49<00:00, 24.89s/it]


 Comparando vetores por similaridade...


100%|██████████| 9310/9310 [29:20<00:00,  5.29it/s] 


 Total de mensagens finais: 9220
  Mensagens removidas por similaridade: 90

Total de pares similares encontrados: 90


In [10]:
df_pares_similares.head()

,mensagem_base,frequencia_base,mensagem_similar,frequencia_similar,similaridade
0,noticia deixa triste kwaivideo,2543,noticia deixa triste kwaivideo,147,1.0
1,acesse link entrar grupo whatsapp chat,209,acesse link entrar grupo whatsapp chat,38,1.0
2,acesse link entrar grupo whatsapp chat,209,acesse link entrar grupo whatsapp chat,32,1.0
3,acesse link entrar grupo whatsapp chat,209,acesse link entrar grupo whatsapp chat,11,1.0
4,acesse link entrar grupo whatsapp chat,209,acesse link entrar grupo whatsapp chat,14,1.0


## Terceira rodada

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
import numpy as np

# --- Codificando os textos ---
modelo_bert = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

df_segunda_rodada = df_segunda_rodada.reset_index(drop=True)

batch_size = 5000
vetores = []
print("Gerando embeddings em batches...")
for i in tqdm(range(0, len(df_segunda_rodada), batch_size)):
    batch_textos = df_segunda_rodada['text_processed'].iloc[i:i + batch_size].tolist()
    batch_vetores = modelo_bert.encode(batch_textos, show_progress_bar=False)
    vetores.extend(batch_vetores)

df_segunda_rodada['vector'] = vetores



# Parâmetro de similaridade
limiar_similaridade = 0.90
janela = 9000  # reduz custo computacional

# Inicialização
indices_para_remover = set()
agrupamento = defaultdict(list)

print(" Comparando vetores por similaridade...")

# Comparar vetores usando cosine_similarity em janela
for i in tqdm(range(len(df_segunda_rodada))):
    if i in indices_para_remover:
        continue
    vec_i = np.array(df_segunda_rodada.iloc[i]['vector']).reshape(1, -1)
    for j in range(i + 1, min(i + janela, len(df_segunda_rodada))):
        if j in indices_para_remover:
            continue
        vec_j = np.array(df_segunda_rodada.iloc[j]['vector']).reshape(1, -1)
        sim = cosine_similarity(vec_i, vec_j)[0][0]
        if sim >= limiar_similaridade:
            agrupamento[i].append(j)
            indices_para_remover.add(j)

# Soma frequência das mensagens agrupadas
linhas_resultado = []
mensagens_utilizadas = set()

for idx_base, similares_idxs in agrupamento.items():
    if idx_base in mensagens_utilizadas:
        continue
    freq_total = df_segunda_rodada.iloc[idx_base]['frequencia']
    mensagens_utilizadas.add(idx_base)

    for sim_idx in similares_idxs:
        freq_total += df_segunda_rodada.iloc[sim_idx]['frequencia']
        mensagens_utilizadas.add(sim_idx)

    linha = df_segunda_rodada.iloc[idx_base].copy()
    linha['frequencia'] = freq_total
    linhas_resultado.append(linha)

# Mensagens que não foram agrupadas
indices_restantes = set(range(len(df_segunda_rodada))) - mensagens_utilizadas
df_restantes = df_segunda_rodada.iloc[list(indices_restantes)].copy()

# Resultado final
df_resultado = pd.DataFrame(linhas_resultado)
df_final_total = pd.concat([df_resultado, df_restantes], ignore_index=True)

print(f"\n Total de mensagens finais: {len(df_final_total)}")
print(f"  Mensagens removidas por similaridade: {len(indices_para_remover)}")

# --- Criar DataFrame com mensagens similares lado a lado ---
pares_similares = []

for idx_base, similares_idxs in agrupamento.items():
    texto_base = df_segunda_rodada.at[idx_base, 'text_processed']
    vetor_base = df_segunda_rodada.at[idx_base, 'vector']
    freq_base = df_segunda_rodada.at[idx_base, 'frequencia']

    for sim_idx in similares_idxs:
        texto_similar = df_segunda_rodada.at[sim_idx, 'text_processed']
        vetor_similar = df_segunda_rodada.at[sim_idx, 'vector']
        freq_similar = df_segunda_rodada.at[sim_idx, 'frequencia']
        similaridade = cosine_similarity(
            np.array(vetor_base).reshape(1, -1),
            np.array(vetor_similar).reshape(1, -1)
        )[0][0]

        if similaridade >= limiar_similaridade:
          pares_similares.append({
                'mensagem_base': texto_base,
                'frequencia_base': freq_base,
                'mensagem_similar': texto_similar,
                'frequencia_similar': freq_similar,
                'similaridade': round(similaridade, 4)
            })

# Criar o DataFrame final com os pares similares
df_pares_similares = pd.DataFrame(pares_similares)

print(f"\nTotal de pares similares encontrados: {len(df_pares_similares)}")



Gerando embeddings em batches...


100%|██████████| 2/2 [00:51<00:00, 25.77s/it]


 Comparando vetores por similaridade...


 11%|█         | 1033/9310 [34:46<3:43:05,  1.62s/it]  